In [ ]:
# Play with PyBaseball - player game data each row is a game
from pybaseball import statcast_batter
from pybaseball import playerid_lookup

import pandas as pd
from datetime import datetime, timedelta


In [249]:
# DATA COLLECTION FOR SINGLE GAME
import pybaseball
from datetime import datetime, timedelta

def get_player_data(player_id, date, dataframe):
    '''
    Description: Based on the inputted information, add data to the given dataframe, 
                 or create one if necessary. The function will RETURN a NEW dataframe
    Parameters:
                player_name: name of the player you want to get data on
                             format should be: last_name, first_name
                date: date you want to search for game data on. 
                            format should be a string 'YYYY-MM-DD'
                dataframe: Pandas dataframe to either begin, or add rows too once data has been cleaned 

    Returns: result_dataframe: dataframe with new information, either created, or appended to paramater dataframe
    '''

    # Enable cache for previous requests
    pybaseball.cache.enable()
    # Set date variable
    # Get data within a date range
    data = statcast_batter(start_dt=date, end_dt=date, player_id=player_id)

    # for column in data.columns:
    #     print(column)
    # Possible relevant columns: pitcher, home_team (for stadium), home_win_exp, starting pitcher, what starting pitcher throws R/L

    # Filter relevant columns for batting performance
    batting_cols = [
        'game_date', 'batter', 'batter','player_name', 'game_pk', 'events', 'at_bat_number',
        'pitch_type', 'balls', 'strikes', 'pitcher', 'p_throws', 'at_bat_number', 
        'pitch_number', 'home_team', 'batter_days_since_prev_game', "inning_topbot"
    ]

    # Get batting subset of data
    batting_data = data[batting_cols].copy()

    if len(batting_data) < 1:
        print(f"No game data found for {player_id} on {date} ")
        return dataframe
    else: 

        # Reset index to avoid duplicate label issues
        batting_data = batting_data.reset_index(drop=True)

        # Remove duplicate columns if they exist (Fixed issues)
        batting_data = batting_data.loc[:, ~batting_data.columns.duplicated()]

        # Get minimum at bat number
        min_at_bat = batting_data['at_bat_number'].min()
        min_at_bat = int(min_at_bat.iloc[0]) if isinstance(min_at_bat, pd.Series) else int(min_at_bat)

        # GET ROW with minimum at_bat_number AND pitch_number == 1
        min_row = batting_data[(batting_data['at_bat_number'] == min_at_bat) & 
                            (batting_data['pitch_number'] == 1)]

        # Establish variables for game info
        starting_pitcher_id = None
        starting_pitcher_throws = None
        home_team = None
        batter_days_since_last_game = None

        # Get game info on starting pitcher and home team
        if not min_row.empty:
            min_row = min_row.iloc[0]  # Get the first row if multiple matches
            home_team = min_row['home_team']
            starting_pitcher_id = min_row['pitcher']
            starting_pitcher_throws = min_row['p_throws']
            batter_days_since_last_game = min_row['batter_days_since_prev_game']
            player_name = min_row['player_name']
            inning_topbot = min_row['inning_topbot']
        else:
            print("No row found with min at_bat_number and pitch_number == 1")

        # Get is_hit column, 1 if player made a hit at bat
        batting_data['is_hit'] = batting_data['events'].isin(['single', 'double', 'triple', 'home_run']).astype(int)
        # Make is_homerun column
        batting_data['is_home_run'] = batting_data['events'].isin(['home_run']).astype(int)
        # Group by for each indivdual at bat
        per_game_at_bat_stats = batting_data.groupby(['game_date', 'game_pk', 'at_bat_number']).agg(
            {
                'is_hit': 'sum', # Total hits
                'is_home_run': 'sum' # Total number of home runs
            }
        ).reset_index()

        # Group by for each GAME
        single_game_outcome = per_game_at_bat_stats.groupby(['game_date', 'game_pk']).agg(
            hits=('is_hit', 'sum'),
            at_bats=('is_hit', 'count'),
            home_runs=('is_home_run', 'sum')
        ).reset_index()

        # Add game info to player's outcome
        single_game_outcome['player_name'] = player_name
        single_game_outcome['home_team'] = home_team
        single_game_outcome['is_home'] = 1 if inning_topbot == 'Bot' else 0
        single_game_outcome['starting_pitcher_id'] = starting_pitcher_id
        single_game_outcome['starting_pitcher_throws'] = starting_pitcher_throws 
        single_game_outcome['batter_days_since_prev_game'] = batter_days_since_last_game

        if dataframe is None or dataframe.empty:
            # If dataframe is empty or None, assign the new data
            result_dataframe = single_game_outcome.copy()
            #print(f"Created new dataframe with {len(result_dataframe)} rows")
        else:
            # If dataframe has data, append the new data
            result_dataframe = pd.concat([dataframe, single_game_outcome], ignore_index=True)
        
        return result_dataframe

        # Feature engineering, home team, starting pitcher throws, starting pitcher 
        # Add park/weather information?
        # add if player is home or away
        # starting pitcher era?
        # add confidence level and check for multiple pitchers



### Current Data Setup For a Batter's Single Game Outcome

game_date  | game_pk | hits | at_bats | home_runs |  player_name home_team  | is_home | starting_pitcher_id | starting_pitcher_throws  | batter_days_since_prev_game  


In [320]:
import pandas as pd
import numpy as np
from statsmodels.genmod.families import Poisson
from statsmodels.genmod.families import NegativeBinomial
from statsmodels.genmod.generalized_linear_model import GLM
from scipy.stats import poisson


# Gather important features and train and run model

def prepare_features(df):
    '''
    Prepare features from df for further modeling.
    Adds recent, season, and career stats, and splits vs. pitcher handedness.
    '''
    df = df.copy()  # Work on a copy to avoid modifying original
    df = df.sort_values(by=['player_name', 'game_date'])  # Sort by player AND date
    
    # Recent performance (last 5/10 games)
    df['hits_last5'] = df.groupby('player_name')['hits'].transform(lambda x: x.shift(1).rolling(5, min_periods=1).sum())
    df['at_bats_last5'] = df.groupby('player_name')['at_bats'].transform(lambda x: x.shift(1).rolling(5, min_periods=1).sum())
    df['avg_hits_last5'] = df['hits_last5'] / 5
    df['batting_avg_last5'] = df['hits_last5'] / df['at_bats_last5']
    
    df['hits_last10'] = df.groupby('player_name')['hits'].transform(lambda x: x.shift(1).rolling(10, min_periods=1).sum())
    df['at_bats_last10'] = df.groupby('player_name')['at_bats'].transform(lambda x: x.shift(1).rolling(10, min_periods=1).sum())
    df['avg_hits_last10'] = df['hits_last10'] / 10
    df['batting_avg_last10'] = df['hits_last10'] / df['at_bats_last10']
    
    # Season stats (to date)
    df['season_hits'] = df.groupby(['player_name', df['game_date'].str[:4]])['hits'].transform(lambda x: x.shift(1).cumsum())
    df['season_at_bats'] = df.groupby(['player_name', df['game_date'].str[:4]])['at_bats'].transform(lambda x: x.shift(1).cumsum())
    df['season_batting_avg'] = df['season_hits'] / df['season_at_bats']
    
    # Career stats (to date)
    df['career_hits'] = df.groupby('player_name')['hits'].transform(lambda x: x.shift(1).cumsum())
    df['career_at_bats'] = df.groupby('player_name')['at_bats'].transform(lambda x: x.shift(1).cumsum())
    df['career_batting_avg'] = df['career_hits'] / df['career_at_bats']
    
    # FIXED: Splits vs. pitcher handedness (last 20 games)
    # Calculate stats against lefties and righties separately, then combine
    
    # Initialize columns
    df['hits_vs_left_last20'] = 0.0
    df['at_bats_vs_left_last20'] = 0.0
    df['hits_vs_right_last20'] = 0.0
    df['at_bats_vs_right_last20'] = 0.0
    
    # Calculate for each player separately
    for player in df['player_name'].unique():
        player_mask = df['player_name'] == player
        player_data = df[player_mask].copy()
        
        # Stats vs lefties
        left_mask = player_data['starting_pitcher_throws'] == 'L'
        if left_mask.sum() > 0:  # Only if player has faced lefties
            left_hits = player_data.loc[left_mask, 'hits'].shift(1).rolling(20, min_periods=1).sum()
            left_abs = player_data.loc[left_mask, 'at_bats'].shift(1).rolling(20, min_periods=1).sum()
            df.loc[player_mask & (df['starting_pitcher_throws'] == 'L'), 'hits_vs_left_last20'] = left_hits
            df.loc[player_mask & (df['starting_pitcher_throws'] == 'L'), 'at_bats_vs_left_last20'] = left_abs
        
        # Stats vs righties
        right_mask = player_data['starting_pitcher_throws'] == 'R'
        if right_mask.sum() > 0:  # Only if player has faced righties
            right_hits = player_data.loc[right_mask, 'hits'].shift(1).rolling(20, min_periods=1).sum()
            right_abs = player_data.loc[right_mask, 'at_bats'].shift(1).rolling(20, min_periods=1).sum()
            df.loc[player_mask & (df['starting_pitcher_throws'] == 'R'), 'hits_vs_right_last20'] = right_hits
            df.loc[player_mask & (df['starting_pitcher_throws'] == 'R'), 'at_bats_vs_right_last20'] = right_abs
    
    # Create the batting average vs current pitcher handedness
    # This is the key feature - performance against the type of pitcher they're facing TODAY
    df['batting_avg_vs_current_hand_last20'] = np.where(
        df['starting_pitcher_throws'] == 'L',
        np.where(df['at_bats_vs_left_last20'] > 0, 
                df['hits_vs_left_last20'] / df['at_bats_vs_left_last20'], 
                df['career_batting_avg']),  # Use career avg as fallback
        np.where(df['at_bats_vs_right_last20'] > 0,
                df['hits_vs_right_last20'] / df['at_bats_vs_right_last20'],
                df['career_batting_avg'])   # Use career avg as fallback
    )
    
    # Pitcher handedness
    df['pitcher_left'] = (df['starting_pitcher_throws'] == 'L').astype(int)
    
    # UPDATED: Features to use (using the new combined feature)
    features = [
        'avg_hits_last5',
        'batting_avg_last5',
        'avg_hits_last10',
        'batting_avg_last10',
        'season_batting_avg',
        'career_batting_avg',
        'batting_avg_vs_current_hand_last20',  # This replaces the problematic feature
        'pitcher_left',
        'batter_days_since_prev_game'
    ]
    
    print("NaN counts before cleaning:")
    nan_counts = df[features + ['hits']].isnull().sum()
    print(nan_counts[nan_counts > 0])
    
    # More intelligent NaN handling
    # Fill remaining NaNs with reasonable defaults
    df['batting_avg_vs_current_hand_last20'] = df['batting_avg_vs_current_hand_last20'].fillna(df['career_batting_avg'])
    df['season_batting_avg'] = df['season_batting_avg'].fillna(df['career_batting_avg'])
    df['career_batting_avg'] = df['career_batting_avg'].fillna(0.250)  # League average fallback
    df['batter_days_since_prev_game'] = df['batter_days_since_prev_game'].fillna(1.0)  # Assume 1 day rest
    
    # For the recent stats, fill with career stats or reasonable defaults
    for col in ['batting_avg_last5', 'batting_avg_last10']:
        df[col] = df[col].fillna(df['career_batting_avg'])
    
    for col in ['avg_hits_last5', 'avg_hits_last10']:
        df[col] = df[col].fillna(1.0)  # Reasonable default
    
    print(f"\nNaN counts after intelligent filling:")
    nan_counts_after = df[features + ['hits']].isnull().sum()
    print(nan_counts_after[nan_counts_after > 0])
    
    # Now drop rows with remaining NaNs (should be very few)
    initial_rows = len(df)
    df_clean = df.dropna(subset=features + ['hits'])
    final_rows = len(df_clean)
    
    print(f"\nRows before cleaning: {initial_rows}")
    print(f"Rows after cleaning: {final_rows}")
    print(f"Rows dropped: {initial_rows - final_rows}")
    
    return df_clean, features


def train_model(df, features):
    print(df.shape)
    X = df[features]
    y = df['hits']
    model = GLM(y, X, family=NegativeBinomial()).fit()
    return model

def calculate_features_for_prediction(df, player_name, game_date, home_team, starting_pitcher_throws, batter_days_since_prev_game):
    """
    Calculate features for a specific player and game based on historical data.
    """
    df_player = df[(df['player_name'] == player_name) & (df['game_date'] < game_date)]
    if df_player.empty:
        raise ValueError(f"No historical data for {player_name} before {game_date}")

    # Recent
    last5 = df_player.tail(5)
    last10 = df_player.tail(10)
    last20_left = df_player[(df_player['starting_pitcher_throws'] == 'L')].tail(20)

    avg_hits_last5 = last5['hits'].mean() if not last5.empty else 0
    batting_avg_last5 = last5['hits'].sum() / last5['at_bats'].sum() if last5['at_bats'].sum() > 0 else 0
    avg_hits_last10 = last10['hits'].mean() if not last10.empty else 0
    batting_avg_last10 = last10['hits'].sum() / last10['at_bats'].sum() if last10['at_bats'].sum() > 0 else 0

    # Season
    season = game_date[:4]
    season_df = df_player[df_player['game_date'].str[:4] == season]
    season_hits = season_df['hits'].sum()
    season_at_bats = season_df['at_bats'].sum()
    season_batting_avg = season_hits / season_at_bats if season_at_bats > 0 else 0

    # Career
    career_hits = df_player['hits'].sum()
    career_at_bats = df_player['at_bats'].sum()
    career_batting_avg = career_hits / career_at_bats if career_at_bats > 0 else 0

    # Splits vs. left
    batting_avg_vs_left_last20 = (
        last20_left['hits'].sum() / last20_left['at_bats'].sum()
        if last20_left['at_bats'].sum() > 0 else 0
    )

    features_row = {
        'avg_hits_last5': avg_hits_last5,
        'batting_avg_last5': batting_avg_last5,
        'avg_hits_last10': avg_hits_last10,
        'batting_avg_last10': batting_avg_last10,
        'season_batting_avg': season_batting_avg,
        'career_batting_avg': career_batting_avg,
        'batting_avg_vs_left_last20': batting_avg_vs_left_last20,
        'pitcher_left': 1 if starting_pitcher_throws == 'L' else 0,
        'batter_days_since_prev_game': batter_days_since_prev_game
    }
    return features_row

def predict_over_under(model, features_row, L):
    """
    Predict probabilities of going over or under a given line L.
    """
    X_new = pd.DataFrame([features_row], columns=features_row.keys())
    lambda_pred = model.predict(X_new)[0]
    K = np.floor(L)
    P_under = poisson.cdf(K, lambda_pred)
    P_over = 1 - P_under
    return P_over, P_under


# Run code here:
# GET DF OF GAMES DATA

def predict_hits(df, player_name, game_date, home_team, starting_pitcher_throws, batter_days_since_last_game, line_for_hits):
    '''
    Input dataframe with data, and player information you want to get information from
    '''

    df = df
    # Prepare features
    df_clean, features = prepare_features(df)

    print(f"Training on {len(df_clean)} rows")
    # Check Df
    if df_clean.empty:
        raise ValueError("No data available after feature preparation. Check for missing values in your features.")
    # Train model
    model = train_model(df_clean, features)
    print(f"It has been {batter_days_since_last_game} since his last game")
    # Get features
    features_row = calculate_features_for_prediction(
            df, player_name, game_date, home_team,
            starting_pitcher_throws, batter_days_since_last_game
        )

    P_over, P_under = predict_over_under(model, features_row, line_for_hits)
    print(f"For {player_name} on {game_date}, with line {line_for_hits}:")
    print(f"Probability of over: {P_over:.4f}")
    print(f"Probability of under: {P_under:.4f}")
    print(f"Prediction: {'Over' if P_over > P_under else 'Under'} with confidence {max(P_over, P_under):.4f}")



In [254]:
from pybaseball import statcast_batter
from pybaseball import playerid_lookup

# find David Ortiz's player id (mlbam_key)
id = playerid_lookup('judge','aaron')
id = id['key_mlbam'][0] 

df = pd.DataFrame()

data_df = get_player_data(id,'2025-05-05', df)
print("This is the data")
print(data_df)



Gathering Player Data
This is the data
    game_date  game_pk  hits  at_bats  home_runs   player_name home_team  \
0  2025-05-05   778045     0        4          0  Judge, Aaron       NYY   

   is_home  starting_pitcher_id starting_pitcher_throws  \
0        1               601713                       R   

   batter_days_since_prev_game  
0                            1  


In [ ]:
from datetime import datetime, timedelta
id = playerid_lookup('judge','aaron')
id = id['key_mlbam'][0] 

judge_df = pd.DataFrame()

# --- ALL DATA FOR AARON JUDGE --- 
start_year = 2016
end_year = 2025
judge_df = pd.DataFrame()  # Initialize your dataframe

for year in range(start_year, end_year + 1):
    for month in range(4, 12):  # April (4) through November (11)
        # Skip if we're past our end date
        if year == 2025 and month > 6:
            break
            
        # Start from August if it's 2016
        if year == 2016 and month < 8:
            continue
            
        # Get first day of the month
        date = f"{year}-{month:02d}-01"
        
        # Iterate through each day of the month
        date_obj = datetime.strptime(date, "%Y-%m-%d")
        while date_obj.month == month and date_obj.strftime("%Y-%m-%d") < '2025-06-19':
            date_str = date_obj.strftime("%Y-%m-%d")
            print(f"Getting data for {date_str}")
            judge_df = get_player_data(id, date_str, judge_df)
            date_obj += timedelta(days=1)


Getting data for 2016-08-01
Gathering Player Data
No game data found for 592450 on 2016-08-01 
Getting data for 2016-08-02
Gathering Player Data
No game data found for 592450 on 2016-08-02 
Getting data for 2016-08-03
Gathering Player Data
No game data found for 592450 on 2016-08-03 
Getting data for 2016-08-04
Gathering Player Data
No game data found for 592450 on 2016-08-04 
Getting data for 2016-08-05
Gathering Player Data
No game data found for 592450 on 2016-08-05 
Getting data for 2016-08-06
Gathering Player Data
No game data found for 592450 on 2016-08-06 
Getting data for 2016-08-07
Gathering Player Data
No game data found for 592450 on 2016-08-07 
Getting data for 2016-08-08
Gathering Player Data
No game data found for 592450 on 2016-08-08 
Getting data for 2016-08-09
Gathering Player Data
No game data found for 592450 on 2016-08-09 
Getting data for 2016-08-10
Gathering Player Data
No game data found for 592450 on 2016-08-10 
Getting data for 2016-08-11
Gathering Player Data


In [258]:
# Features to Add
# Who is the opponent
# Pitcher ERA?
judge_df.to_csv('aaron_judge_data.csv')

In [ ]:
# Pick 10 players, get their csv, set up an airflow to add their information daily
# Update csv with all dates before current date
# Send me a daily result for their game and their info for their game today

# Players to use:
# Aaron Judge, Shohei Ohtani, Juan Soto, Gunnar Henderson, Bobby Witt Jr, Kyle Tucker, Kyle Schwaber, Pete Alonso, Manny Machado 

# Need to get game data to pass into the function, Web scraping
# Need to get: player_name, game_date, home_team, starting_pitcher_throws, batter_days_since_last_game, line_for_hits



In [262]:
csv = 'aaron_judge_data.csv'
id = playerid_lookup('judge','aaron')
player_id = id['key_mlbam'][0] 

def update_csv(csv, player_id):

    # Read the CSV file
    try:
        df = pd.read_csv(csv)
        print(f"Successfully read CSV with {len(df)} rows")
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return None
    
    # Convert game_date to datetime
    try:
        df['game_date'] = pd.to_datetime(df['game_date'])
        print("Successfully converted game_date to datetime")
    except Exception as e:
        print(f"Error converting game_date to datetime: {e}")
        return None
    
    # Find the most recent date in the CSV
    if df.empty or df['game_date'].isna().all():
        print("No valid dates found in game_date column")
        # Start from 30 days ago if no data exists
        most_recent_date = datetime.now() - timedelta(days=30)
        print(f"Starting from 30 days ago: {most_recent_date.date()}")
    else:
        most_recent_date = df['game_date'].max()
        print(f"Most recent date in CSV: {most_recent_date.date()}")
        
        # Start from the day after the most recent date
        most_recent_date = most_recent_date + timedelta(days=1)
        print(f"Starting update from: {most_recent_date.date()}")
    
    # Get today's date
    today = datetime.now()
    print(f"Today's date: {today.date()}")
    
    # Generate list of dates to process
    dates_to_process = []
    current_date = most_recent_date

    while current_date.date() <= today.date():
        dates_to_process.append(current_date)
        current_date += timedelta(days=1)
    
    print(f"Need to process {len(dates_to_process)} dates")
    
    if not dates_to_process:
        print("No new dates to process - data is already up to date!")
        return df
    
    # Process each date
    for i, date in enumerate(dates_to_process):
        print(f"Processing date {i+1}/{len(dates_to_process)}: {date.date()}")
        try:
            # Call your get_player_data function
            df = get_player_data(player_id, date.strftime('%Y-%m-%d'), df)
                   
        except Exception as e:
            print(f"  Error processing {date.date()}: {e}")
            continue

    return df

df = update_csv(csv, player_id)

Successfully read CSV with 1112 rows
Successfully converted game_date to datetime
Most recent date in CSV: 2025-06-17
Starting update from: 2025-06-18
Today's date: 2025-06-20
Need to process 3 dates
Processing date 1/3: 2025-06-18
Gathering Player Data
Processing date 2/3: 2025-06-19
Gathering Player Data
Processing date 3/3: 2025-06-20
Gathering Player Data
No game data found for 592450 on 2025-06-20 


In [321]:
# Check aaron judge for tn
predict_hits(judge_df, "Judge, Aaron", "2025-06-20", "NYY", "R", 0, 0.5)

NaN counts before cleaning:
avg_hits_last5                         1
batting_avg_last5                      1
avg_hits_last10                        1
batting_avg_last10                     1
season_batting_avg                    10
career_batting_avg                     1
batting_avg_vs_current_hand_last20     1
dtype: int64

NaN counts after intelligent filling:
season_batting_avg                    1
batting_avg_vs_current_hand_last20    1
dtype: int64

Rows before cleaning: 1102
Rows after cleaning: 1101
Rows dropped: 1
Training on 1101 rows
(1101, 33)
It has been 0 since his last game
For Judge, Aaron on 2025-06-20, with line 0.5:
Probability of over: 0.6592
Probability of under: 0.3408
Prediction: Over with confidence 0.6592


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [ ]:
# Change from date object to date string
def fix_dataframe_for_string_operations(df):
    """
    Quick fix to ensure game_date can be used with string operations like .str[:4]
    """
    df_fixed = df.copy()
    
    if 'game_date' in df_fixed.columns:
        # Convert to string format that works with your existing code
        if pd.api.types.is_datetime64_any_dtype(df_fixed['game_date']):
            # Convert datetime to string in format that matches your original data
            df_fixed['game_date'] = df_fixed['game_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
        
        # Ensure it's string type
        df_fixed['game_date'] = df_fixed['game_date'].astype(str)
    
    return df_fixed

df = fix_dataframe_for_string_operations(df)
df.head(5)

,Unnamed: 0,game_date,game_pk,hits,at_bats,home_runs,player_name,home_team,is_home,starting_pitcher_id,starting_pitcher_throws,batter_days_since_prev_game
0,0.0,2016-08-13 00:00:00,448611,2,4,1,"Judge, Aaron",NYY,1,542882,R,NaN
1,1.0,2016-08-14 00:00:00,448626,1,4,1,"Judge, Aaron",NYY,1,543606,R,1.0
2,2.0,2016-08-15 00:00:00,448633,2,4,0,"Judge, Aaron",NYY,1,285079,R,1.0
3,3.0,2016-08-16 00:00:00,448648,0,4,0,"Judge, Aaron",NYY,1,462136,R,1.0
4,4.0,2016-08-17 00:00:00,448663,2,4,0,"Judge, Aaron",NYY,1,457918,L,1.0


In [298]:
judge_df = pd.read_csv("aaron_judge_data.csv")